# WARNING

The preview for this file in Github is broken. **DO NOT copy and paste from the preview.** Read the README for further instructions.

# Extract, Transform

In [ ]:
import requests
import pandas as pd
import numpy as np
import gc

willStore = False
lines = []

for line in requests.get('https://creatures-of-sonaria-official.fandom.com/wiki/Module:CreatureData/data').text.split('\n'):
    line = line.strip()
    if line == '' or '<meta' in line:
        willStore = False
    if willStore:
        lines.append(line)
    if 'p.creatureStats' in line:
        willStore = True

strings = ''.join(lines).replace('&#039;','').replace('&quot;','"').replace('&lt;','').replace('&gt;','').split('\t')[:-1]

columns = ['name']

for string in strings:
    if '}' in string:
        break
    if '{' not in string:
        columns.append(string[1:string.index(']')])

data = [[]]
index = 0

for string in strings:
    #preliminary data cleaning
    string = string.replace('Block Poison (0.3) Iron Stomach','Block Poison (0.3), Iron Stomach').replace('Forstbite','Frostbite').replace(').','),').replace('  ', ' ')

    if '[[' in string:
        string = string[:string.index('[[')] + string[string.rindex(']]')+2:]
    if '}' in string:
        index += 1
        data.append([])
    if '"' in string:
        temp = string[string.index('"')+1:string.rindex('"')].strip()
        if temp.endswith('min'):
            temp = temp[:temp.index('min')-1]
        if temp.isdecimal():
            temp = int(temp)
        elif temp.replace('.','').isdecimal():
            temp = float(temp)
        elif temp == 'N/A':
            temp = np.nan
        data[index].append(temp)
    else:
        data[index].append(string[string.index('/')-2:string.rindex(',')])

df = pd.DataFrame(data, columns=columns)

if all(df['name'] == df['common']):
    df.drop(columns='common', inplace=True)
df.drop(columns=[col for col in df.columns if df[col].nunique(dropna=False) == 1], inplace=True) #drops hungerDrain and thirstDrain

eWeightList = []
eDPSList = []
eHPList = []
for i in range(df.shape[0]):
    eWeight = 1
    eDPS = 1
    eHP = 1
    if isinstance(df.at[i, 'activated'], str):
        if 'Fortify' in df.at[i, 'activated']:
            eWeight += 0.05
        if 'Harden' in df.at[i, 'activated']:
            eWeight += 0.25
        if 'Adrenaline' in df.at[i, 'activated']:
            eDPS += 0.2
        if 'Unbridled Rage' in df.at[i, 'activated']:
            eDPS += 0.3
        if 'Wardens Rage' in df.at[i, 'activated']:
            eDPS += 5.625
        if 'Cause Fear' in df.at[i, 'activated']:
            eHP += 1
    if isinstance(df.at[i, 'passive'], str) and 'Berserk' in df.at[i, 'passive']:
        eDPS += 0.2
    eWeight *= df.at[i, 'weight']
    eWeightList.append(eWeight)
    eDPS *= df.at[i, 'damage'] * (1 + eWeight / 50000) / (2 * df.at[i, 'biteCooldown'])
    eDPSList.append(eDPS)
    eHP *= df.at[i, 'health'] / (1 + 50000 / eWeight)
    eHPList.append(eHP)
df['effectiveWeight'] = eWeightList
df['effectiveDPS'] = eDPSList
df['effectiveHP'] = eHPList
df['effectivePower'] = df['effectiveDPS'] * df['effectiveHP']

def flySprintMultiplier(fsm: float) -> float:
    if np.isnan(fsm):
        return 1
    return 1 + fsm
def ambush(a: float) -> float:
    if np.isnan(a):
        return 1
    return a
def agileSwimmer(passive) -> float:
    if isinstance(passive, str) and 'Agile Swimmer' in passive:
        return 1.75
    return 1
df['flySprintSpeed'] = df['flySpeed'] * df['flySprintMultiplier'].apply(flySprintMultiplier)
df['swimSprintSpeed'] = df['sprintSpeed'] * df['passive'].apply(agileSwimmer)
df['ambushSpeed'] = df['sprintSpeed'] * df['ambush'].apply(ambush)

def wardensRage(string):
    if isinstance(string, str):
        return string.replace('Wardens Rage', "Warden's Rage")
    return string
df['activated'] = df['activated'].apply(wardensRage)

def split(string) -> list[str]:
    if not isinstance(string, str):
        return []
    list1 = string.split(',')
    for string1 in list1:
        string1 = string1.strip()
    return list1
passives = df['passive'].apply(split)
activateds = df['activated'].apply(split)
tempDF = pd.merge(df['name'], passives, left_index=True, right_index=True)
passiveDF = tempDF.explode('passive', ignore_index=True)
tempDF = pd.merge(df['name'], activateds, left_index=True, right_index=True)
activatedDF = tempDF.explode('activated', ignore_index=True)

spreadsheetDF = df.copy()
df.drop(columns=['passive', 'activated'], inplace=True)

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()
for col in spreadsheetDF.columns:
    if spreadsheetDF[col].dtype == 'object':
        spreadsheetDF[col] = spreadsheetDF[col].str.strip()
for col in passiveDF.columns:
    passiveDF[col] = passiveDF[col].str.strip()
for col in activatedDF.columns:
    activatedDF[col] = activatedDF[col].str.strip()

del willStore, lines, strings, line, columns, data, index, temp, string, eWeightList, eDPSList, eHPList, eWeight, eDPS, eHP, i, flySprintMultiplier, ambush, agileSwimmer, wardensRage, split, passives, activateds, tempDF, col
gc.collect()

432

# Verify

In [ ]:
#This code produces 7 different outputs: Whether names are unique, suspicious flyers, unique passive abilities,
#unique activated abilities, miscategorized abilities, rows in spreadsheetDF with trailing commas in passive or activated,
#and number of missing values in each column.
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 500)

print('Names are unique:')
print(df['name'].nunique() == df.shape[0])
print('\nSuspicious flyers:')
df.reset_index(drop=True, inplace=True)
for i in range(df.shape[0]):
    if not np.isnan(df.at[i, 'flySpeed']):
        if np.isnan(df.at[i, 'flySprintMultiplier']) or np.isnan(df.at[i, 'glideStaminaRegen']) or np.isnan(df.at[i, 'takeoffMultiplier']):
            print(df.loc[i, ['name', 'flySpeed', 'flySprintMultiplier', 'glideStaminaRegen', 'takeoffMultiplier']])

def fillnan(string) -> str:
    if isinstance(string, str):
        return string
    return ''
print('\nUnique passive abilities:')
print(np.sort(passiveDF['passive'].apply(fillnan).unique()))
print('\nUnique activated abilities:')
print(np.sort(activatedDF['activated'].apply(fillnan).unique()))
print('\nMiscategorized abilities:')
print(pd.merge(passiveDF, activatedDF, left_on='passive', right_on='activated')['passive'].apply(fillnan).unique())
print('\nTrailing commas:')
spreadsheetDF.reset_index(drop=True, inplace=True)
for i in range(spreadsheetDF.shape[0]):
    if isinstance(spreadsheetDF.at[i, 'passive'], str) and spreadsheetDF.at[i, 'passive'].strip().endswith(','):
        print(spreadsheetDF.loc[i, ['name', 'passive']])
    if isinstance(spreadsheetDF.at[i, 'activated'], str) and spreadsheetDF.at[i, 'activated'].strip().endswith(','):
        print(spreadsheetDF.loc[i, ['name', 'activated']])
print('\nNumber of missing values in each column:')
df.isnull().sum()

Names are unique:
True

Suspicious flyers:
name                   Corsarlett
flySpeed                     41.0
flySprintMultiplier           NaN
glideStaminaRegen             2.0
takeoffMultiplier            20.0
Name: 61, dtype: object
name                   Exotide
flySpeed                  48.0
flySprintMultiplier        NaN
glideStaminaRegen          1.0
takeoffMultiplier         10.0
Name: 93, dtype: object
name                   Flamerir
flySpeed                   39.0
flySprintMultiplier         NaN
glideStaminaRegen           1.5
takeoffMultiplier          20.0
Name: 104, dtype: object
name                   Glovaucus
flySpeed                    40.0
flySprintMultiplier          NaN
glideStaminaRegen            1.0
takeoffMultiplier           20.0
Name: 126, dtype: object
name                   Gryvila
flySpeed                  45.0
flySprintMultiplier        NaN
glideStaminaRegen          1.0
takeoffMultiplier         10.0
Name: 133, dtype: object
name                   Squiti

,0
name,0
ambush,188
appetite,0
beachSpeed,336
biteCooldown,0
bloodColor,0
bloodTexture,0
breath,248
type,0
damage,0


# Clean

Checklist:
* Some of the "Breath Resistance" and "Block (Status)" abilities display as 0.1 instead of 0.10, 0.2 instead of 0.20, and so on up to 1.0 instead of 1.00. ✅
* Many "First Strike" amounts and "Toxic Trail" amounts are in decimal form instead of being in whole number form(there's some of each passive in whole number form though). ✅
* Those decimal form amounts also have the same issue as with the "Breath Resistance" and "Block (Status)" abilities. ✅
* There's 3 cases of duplicated spacing (2 spaces instead of 1). ✅
* Danténos's list of passive abilities has a period instead of a comma between "Block Bleed" and "Block Burn". ✅
* Flipli and Nimoona have "Latch" without a number, the other creatures with that ability have "Latch (2)". ✅
* Hygos's list of passive abilities is missing a comma between "Block Poison" and "Iron Stomach". ✅
* Lactarim's "Radiation" ability is missing a number. Lactarim should have "Radiation (0.75)". ✅
* Nindyr, Sarchias, and Shiyaro's "Strength In Numbers" ability isn't capitalized correctly. ✅
* Oxytalis's "Life Leech" ability is missing a number. Oxytalis should have "Life Leech (0.30)". ✅
* Corsarlett, Flamerir, Glovaucus, Gryvila, Squitico, Zhelijin, Zohdok, and Zwevealisk should have a flySprintMultiplier of 0.25 not N/A. This is to distinguish them from Exotide, which is still missing a flySprintMultiplier despite being able to fly. ✅
* Valkurse has the wrong jumpStamina value. It should be 20, not 0. ✅
* Arkmetta and Takamorath are missing nightvision values. Arkmetta's nightvision should be 3 and Takamorath's nightvision should be 1. ✅
* Fluren has "Raider" listed in activated abilities instead of in passive abilities. ✅
* Pernirex, Shro, and Tsugae have "Egg Stealer" listed in passive abilities instead of in activated abilities. ✅
* Slaiorn has the wrong "Radiation" value. It should be "Radiation (0.50)" instead of "Radiation (3)". ✅
* Sylvesqual has "Cursed Sigil" listed in passive abilities instead of in activated abilities. ✅
* Xenolif has the wrong "Radiation" value. It should be "Radiation (0.75)" instead of "Radiation (3)". ✅
* Thykallos has "Frosty" in both passive abilities and activated abilities, instead of being only in passive abilities. ✅
* In Amolis's passives, "Frostbite" is misspelled as "Forstbite". ✅

Note that some of these were addressed in the Extract step.

In [ ]:
#This only works if the names are unique
df.set_index('name', drop=False, inplace=True)
spreadsheetDF.set_index('name', drop=False, inplace=True)

for name in ('Corsarlett', 'Flamerir', 'Glovaucus', 'Gryvila', 'Squitico', 'Zhelijin', 'Zohdok', 'Zwevealisk'):
    df.at[name, 'flySprintMultiplier'] = 0.25
    spreadsheetDF.at[name, 'flySprintMultiplier'] = 0.25
df.at['Valkurse', 'jumpStamina'] = 20
spreadsheetDF.at['Valkurse', 'jumpStamina'] = 20
df.at['Arkmetta', 'nightvision'] = 3
spreadsheetDF.at['Arkmetta', 'nightvision'] = 3
df.at['Takamorath', 'nightvision'] = 1
spreadsheetDF.at['Takamorath', 'nightvision'] = 1

#spreadsheetDF fixes
spreadsheetDF.at['Fluren', 'passive'] = spreadsheetDF.at['Fluren', 'passive'] + ', Raider'
spreadsheetDF.at['Fluren', 'activated'] = spreadsheetDF.at['Fluren', 'activated'].replace('Raider', '').replace(', , ', ', ')
if spreadsheetDF.at['Fluren', 'activated'].startswith(','):
    spreadsheetDF.at['Fluren', 'activated'] = spreadsheetDF.at['Fluren', 'activated'][1:].strip()
elif spreadsheetDF.at['Fluren', 'activated'].strip().endswith(','):
    spreadsheetDF.at['Fluren', 'activated'] = spreadsheetDF.at['Fluren', 'activated'].strip()[:-1]
for name in ('Pernirex', 'Shro', 'Tsugae'):
    spreadsheetDF.at[name, 'activated'] = spreadsheetDF.at[name, 'activated'] + ', Egg Stealer'
    spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'].replace('Egg Stealer', '').replace(', , ', ', ')
    if spreadsheetDF.at[name, 'passive'].startswith(','):
        spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'][1:].strip()
    elif spreadsheetDF.at[name, 'passive'].strip().endswith(','):
        spreadsheetDF.at[name, 'passive'] = spreadsheetDF.at[name, 'passive'].strip()[:-1]
spreadsheetDF.at['Slaiorn', 'activated'] = spreadsheetDF.at['Slaiorn', 'activated'].replace('Radiation (3)', 'Radiation (0.50)')
spreadsheetDF.at['Sylvesqual', 'activated'] = spreadsheetDF.at['Sylvesqual', 'activated'] + ', Cursed Sigil (6)'
spreadsheetDF.at['Sylvesqual', 'passive'] = spreadsheetDF.at['Sylvesqual', 'passive'].replace('Cursed Sigil (6)', '').replace(', , ', ', ')
if spreadsheetDF.at['Sylvesqual', 'passive'].startswith(','):
    spreadsheetDF.at['Sylvesqual', 'passive'] = spreadsheetDF.at['Sylvesqual', 'passive'][1:].strip()
elif spreadsheetDF.at['Sylvesqual', 'passive'].strip().endswith(','):
    spreadsheetDF.at['Sylvesqual', 'passive'] = spreadsheetDF.at['Sylvesqual', 'passive'].strip()[:-1]
spreadsheetDF.at['Xenolif', 'activated'] = spreadsheetDF.at['Xenolif', 'activated'].replace('Radiation (3)', 'Radiation (0.75)')
spreadsheetDF.at['Thykallos', 'activated'] = spreadsheetDF.at['Thykallos', 'activated'].replace('Frosty', '').replace(', , ', ', ')
if spreadsheetDF.at['Thykallos', 'activated'].startswith(','):
    spreadsheetDF.at['Thykallos', 'activated'] = spreadsheetDF.at['Thykallos', 'activated'][1:].strip()
elif spreadsheetDF.at['Thykallos', 'activated'].strip().endswith(','):
    spreadsheetDF.at['Thykallos', 'activated'] = spreadsheetDF.at['Thykallos', 'activated'].strip()[:-1]

#Remake passiveDF & activatedDF
def split(string) -> list[str]:
    if not isinstance(string, str):
        return []
    list1 = string.split(',')
    for string1 in list1:
        string1 = string1.strip()
    return list1
passives = spreadsheetDF['passive'].apply(split)
activateds = spreadsheetDF['activated'].apply(split)
tempDF = pd.merge(spreadsheetDF['name'], passives, left_index=True, right_index=True)
passiveDF = tempDF.explode('passive', ignore_index=True)
tempDF = pd.merge(spreadsheetDF['name'], activateds, left_index=True, right_index=True)
activatedDF = tempDF.explode('activated', ignore_index=True)
for col in passiveDF.columns:
    passiveDF[col] = passiveDF[col].str.strip()
for col in activatedDF.columns:
    activatedDF[col] = activatedDF[col].str.strip()

#More passiveDF & activatedDF fixes
def decimalFormattingFix(string):
    if isinstance(string, str) and 'Attack' not in string:
        if '(0.' in string and string.index(')') - string.index('(') != 5:
            return string.replace(')','0)')
        if '(-0.' in string and string.index(')') - string.index('(') != 6:
            return string.replace(')','0)')
        if '1.0)' in string:
            return string.replace(')','0)')
    return string
def passiveFixes(string):
    if isinstance(string, str):
        if 'First Strike' in string or 'Toxic Trail' in string:
            return string.replace('0.', '')
        if 'Strength in Numbers' in string:
            return string.replace('Strength in Numbers', 'Strength In Numbers')
    return string
def activatedFixes(string):
    if isinstance(string, str):
        if 'Latch' in string and '(' not in string:
            return string.replace('Latch','Latch (2)')
        if 'Life Leech' in string and '(' not in string:
            return string.replace('Life Leech','Life Leech (0.30)')
        if 'Radiation' in string and '(' not in string:
            return string.replace('Radiation','Radiation (0.75)')
    return string
passiveDF['passive'] = passiveDF['passive'].apply(decimalFormattingFix).apply(passiveFixes)
activatedDF['activated'] = activatedDF['activated'].apply(activatedFixes)

# Load

The first 3 tsv files are for importing into a database (although I haven't tested them in a database management system aside from LibreBase which did not work). The last one is for opening with Google Sheets or Microsoft Excel.

In [ ]:
df.to_csv('CreatureData.tsv', sep='\t', na_rep='-1', index=False)
passiveDF.to_csv('CreaturePassive.tsv', sep='\t', index=False)
activatedDF.to_csv('CreatureActivated.tsv', sep='\t', index=False)
spreadsheetDF.to_csv('CreatureDataSpreadsheet.tsv', sep='\t', index=False)